### Combine pkl's of existing SL ROI's with added controls (rather than re-compute all regions over again)

In [1]:
import os
from pathlib import Path
import sys
sys.path.append('/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/')
from eeg_toolkit import utils, preprocess
from eeg_toolkit import source_localization as sl

0 files missing from root.txt in /home/wanglab/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/wanglab/mne_data/MNE-fsaverage-data/fsaverage


### Paths

In [2]:
# Time window for source localization computation
times_tup,time_win_path = preprocess.get_time_window(5)

# Average dipoles, always true unless you have subject-specific MRI and know what you're doing
average_dipoles = True

# Save inverse. True if you want to plot the brain model with AEC connectivity
save_inv = True

[-2.5,0.0,2.5]


In [3]:
# Read in processed raw data
data_dir = Path("../../Data")
processed_data_path = data_dir / 'Processed Data'
epo_path = data_dir / time_win_path

### Subject IDs

In [4]:
# viable subjects
sub_ids = utils.import_subs(os.path.join(data_dir),'sub_ids.txt')
# select only 64ch subs and turn into set
sub_ids = sub_ids[7:]
print(sub_ids)

['018', '020', '021', '022', '023', '024', '027', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', 'C1.', 'C2.', 'C3.', 'C5.', 'C6.', 'C7.', 'C8.', 'C9.', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C24', 'C25', 'C26', 'C27']


In [16]:
# All CP and HC subs
sub_ids_CP = [  
    '018',  
    '020',  
    '021',  
    '027',  
    '031',  
    '032',  
    '033',  
    '035',  
    '036',  
    '037',  
    '038',  
    '040',  
    '043',  
    '044',  
    '048',  
    '049',  
    '050',  
    '057',  
    '059',
]

sub_ids_HC = [
    'C10',  
    'C11',  
    'C12',  
    'C13',  
    'C14',  
    'C15',  
    'C16',  
    'C17',  
    'C18',  
    'C19',  
    'C2.',  
    'C24',  
    'C25',  
    'C26',  
    'C27',  
    'C3.',  
    'C6.',  
    'C7.',  
    'C9.'   
]
 

# CP Subs with widespread pain
sub_ids_WSP = [
    '018',
    '022',
    '024',
    '031',
    '032',
    '034',
    '036',
    '039',
    '040',
    '045',
    '046',
    '052',
]

# CP Subs with localized pain
sub_ids_LP = [  
    '020',  
    '021',  
    '023',  
    '029',  
    '031',  
    '037',  
    '041',  
    '042',  
    '048',  
    '049',  
    '050',  
    '056'  
]  

 
sub_ids = sub_ids_CP + sub_ids_HC + sub_ids_WSP + sub_ids_LP

# keep only unique
sub_ids = sorted(list(set(sub_ids)))

In [6]:
print(f"Chronics: {len([el for el in sub_ids if el.startswith('0')])}")
print(f"Controls: {len([el for el in sub_ids if el.startswith('C')])}")
print(f"Total: {len(sub_ids)}")

Chronics: 31
Controls: 19
Total: 50


In [41]:
# Get stc only from selected labels
roi_names = [# Left
             'rostralanteriorcingulate-lh', # Left Rostral ACC
             'caudalanteriorcingulate-lh', # Left Caudal ACC
             'postcentral-lh', # Left S1,
             'insula-lh', 'superiorfrontal-lh', # Left Insula, Left DL-PFC,
             'medialorbitofrontal-lh', # Left Medial-OFC
             # CONTROLS
             # lateral occipital
             'lateraloccipital-lh', # Left Visual Cortex
             'superiortemporal-lh', # Left Auditory Cortex
             # Right
             'rostralanteriorcingulate-rh', # Right Rostral ACC
             'caudalanteriorcingulate-rh', # Right Caudal ACC
             'postcentral-rh', # , Right S1
             'insula-rh', 'superiorfrontal-rh', # Right Insula, Right DL-PFC
             'medialorbitofrontal-rh', # Right Medial-OFC
             # CONTROLS
             'lateraloccipital-rh', # Right Visual Cortex
             'superiortemporal-rh', # Right Auditory Cortex
]

In [42]:
roi_names_control = [# Left
             # CONTROLS
             # lateral occipital
             'lateraloccipital-lh', # Left Visual Cortex
             'superiortemporal-lh', # Left Auditory Cortex
             'lateraloccipital-rh', # Right Visual Cortex
             'superiortemporal-rh', # Right Auditory Cortex
]

In [45]:
# make a dict with the control roi labels and their index in roi_names
control_roi_ids = dict([(roi_names[i], i) for i in range(len(roi_names)) if roi_names[i] in roi_names_control])

In [47]:
control_roi_ids

{'lateraloccipital-lh': 6,
 'superiortemporal-lh': 7,
 'lateraloccipital-rh': 14,
 'superiortemporal-rh': 15}

In [63]:
nan_subjects=[]

methods = [
    'MNE',
    # 'dSPM',
    ]

for sub_id in sub_ids:    
    for save_stc_mat in [
        False,
        # True,
        ]:    
        for method in methods:
            print(f"Source Localization Method: {method}")                          

            # Paths
            stc_path = (data_dir / f'Source Time Courses ({method}) (MAT)' 
                        if save_stc_mat 
                        else data_dir / f'Source Time Courses ({method})')
                           
            EO_resting_save_path = stc_path / "Eyes Open"
            EC_resting_save_path = stc_path / "Eyes Closed"
            zscored_epochs_save_path = stc_path / "zscored_Epochs" / time_win_path

            # open the pkl's
            print(f"Loading {sub_id} data...")
            # eyes open
            eo_data = utils.unpickle_data(EO_resting_save_path, f"{sub_id}_eyes_open.pkl")
            eo_control = utils.unpickle_data(EO_resting_save_path, f"{sub_id}_eyes_open_control.pkl")
            
            # epochs
            epochs = utils.unpickle_data(zscored_epochs_save_path, f"{sub_id}_epochs.pkl")
            epochs_control = utils.unpickle_data(zscored_epochs_save_path, f"{sub_id}_epochs_control.pkl")

Source Localization Method: MNE
Loading 018 data...
Source Localization Method: MNE
Loading 020 data...


FileNotFoundError: [Errno 2] No such file or directory: '../../Data/Source Time Courses (MNE)/Eyes Open/020_eyes_open_control.pkl'

In [62]:
eo_data.shape

(12, 121815)

In [61]:
eo_control.shape

(4, 141299)

In [66]:
# Insert eo_control data into eo_data using the control_roi_ids dict
eo_data[control_roi_ids['lateraloccipital-lh'],:]=eo_control[:,0]
eo_data[control_roi_ids['superiortemporal-lh'],:]=eo_control[:,1]
eo_data[control_roi_ids['lateraloccipital-rh'],:]=eo_control[:,2]
eo_data[control_roi_ids['superiortemporal-rh'],:]=eo_control[:,3]

ValueError: could not broadcast input array from shape (4,) into shape (121815,)